<a href="https://colab.research.google.com/github/cicl-iscl/LeWiDi_SemEval2023/blob/main/Notebooks/TryMultiplethings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes

ConvAbuse is just the dictionary as a string lol

In [2]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install wandb
!pip install torchsampler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 988.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

# Set up wandb

In [3]:
import wandb
# config = dict (
#   architecture = "bert_arabic_english",
#   epochs = 1,
#   batch_size = 4
# )

# run = wandb.init(
#   project="capture-disagreement",
#   notes="test_run",
#   #name = "test_run",
#   config=config,
#   save_code = True,
#   tags = ["bert_arabic_english", "weighted_loss", "balanced_loss", "only_last_layer"],
#   #magic = True
# )

run = wandb.init(
    project="capture-disagreement",
    save_code = True,
)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Eval scripts

In [4]:
import evaluate

#TODO something is off here, untrained model gets 0.6 im ce eval

def evaluate_f1_score_huggingface(model, eval_dataloader):
  total = 0
  f1 = 0
  f1_metric = evaluate.load("f1")
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, hard_labels = batch["output"].to(device), batch["labels"].to(device)
    with torch.no_grad():
      outputs = model(inputs)
    logits = outputs
    predictions = torch.argmax(logits, dim=-1)
    N = predictions.shape[0]
    total+=1#the metric already gives the average, so no + batch_size
    f1 += f1_metric.compute(predictions=predictions, references=hard_labels, average = 'micro')["f1"]#micro must be used, otherwise smth wrong is returned, TODO look into that (00 00 is undefined)

  return f1/total


#slightly different to the pytorch implementations, not sure why TODO
def evaluate_ce_score(model, eval_dataloader, epsilon = 1e-12):
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, hard_labels = batch["output"].to(device), batch["labels"].to(device)
    soft_labels = torch.cat([batch["soft_labels"]["0"][0].unsqueeze(0), batch["soft_labels"]["1"][0].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(inputs)
    logits = outputs# + 0.5
    #probabilities = logits
    probabilities = torch.softmax(logits, axis=-1)

    #probabilities = torch.softmax(probabilities, axis=-1)
    probabilities = torch.zeros_like(probabilities) + 0.5

    print(soft_labels[-5:])
    print(probabilities[-5:])
    print()
    #break

    predictions = torch.clip(probabilities, epsilon, 1. - epsilon)                                      
    N = predictions.shape[0]
    total+=N#maybe should be 1 here as well
    #ce += -torch.sum(soft_labels*torch.log(predictions+1e-9))
    #print(ce)
    ce += -torch.sum(soft_labels*torch.log(predictions))
  return ce/total


def evaluate_bce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.BCELoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, hard_labels, soft_labels = batch["output"].to(device), batch["labels"].to(device), batch["soft_labels"]["1"][0].to(device)
    #inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device), batch["soft_label"]["1"].to(device)
    with torch.no_grad():
      outputs = model(inputs)
    logits = outputs
    probabilities = torch.softmax(logits, axis=-1)
    N = probabilities.shape[0]
    total +=N
    ce += criterion(probabilities[:,-1], soft_labels)
  return ce/total


def evaluate_ce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.CrossEntropyLoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs,  hard_labels = batch["output"].to(device), batch["labels"].to(device)
    soft_labels = torch.cat([batch["soft_labels"]["0"][0].unsqueeze(0), batch["soft_labels"]["1"][0].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(inputs)
    logits = outputs
    probabilities = logits                         
    N = probabilities.shape[0]
    total+=N
    ce += criterion(probabilities, soft_labels)
  return ce/total



# Load data

In [5]:
bert_multilingual_last_layer_dataset = run.use_artifact('capture_disagreement/capture-disagreement/bert_multilingual_last_layer_dataset:latest', type='dataset')
bert_multilingual_last_layer_dataset_directory = bert_multilingual_last_layer_dataset.download()

wandb: Downloading large artifact bert_multilingual_last_layer_dataset:latest, 265.92MB. 10431 files... 
wandb:   10431 of 10431 files downloaded.  
Done. 0:1:46.7


In [6]:
import torch
import os

class ModelOutputDataset(torch.utils.data.Dataset):
    def __init__(self, output_dir):
        self.output_dir = output_dir
        self.output_paths = [os.path.join(output_dir, path) for path in os.listdir(output_dir)]
        
    def __len__(self):
        return len(self.output_paths)
    
    def __getitem__(self, index):
        output_path = self.output_paths[index]
        data = torch.load(output_path, map_location=device)
        return data

In [7]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
bert_multilingual_last_layer_dataset = ModelOutputDataset(bert_multilingual_last_layer_dataset_directory)

## Balanced

In [ ]:
import torch
import os
from collections import Counter

# Check if PyTorch version is greater than 1.5.0
if int(torch.__version__[0]) > 1 or (int(torch.__version__[0]) == 1 and int(torch.__version__[2]) >= 5):
    from torch.utils.data.sampler import WeightedRandomSampler
else:
    # Implement WeightedRandomSampler using custom code
    class WeightedRandomSampler:
        def __init__(self, weights, num_samples, replacement=True):
            self.weights = torch.DoubleTensor(weights)
            self.num_samples = num_samples
            self.replacement = replacement

        def __iter__(self):
            return iter(torch.multinomial(self.weights, self.num_samples, self.replacement))

        def __len__(self):
            return self.num_samples

class ModelOutputDataset(torch.utils.data.Dataset):
    def __init__(self, output_dir):
        self.output_dir = output_dir
        self.output_paths = [os.path.join(output_dir, path) for path in os.listdir(output_dir)]
        
    def __len__(self):
        return len(self.output_paths)
    
    def __getitem__(self, index):
        output_path = self.output_paths[index]
        data = torch.load(output_path, map_location=device)
        return data

bert_multilingual_last_layer_dataset = ModelOutputDataset(bert_multilingual_last_layer_dataset_directory)

# Split the dataset into train and test
train_set, test_set = torch.utils.data.random_split(bert_multilingual_last_layer_dataset, [0.8, 0.2])
test_set, eval_set = torch.utils.data.random_split(test_set, [0.5, 0.5])

# Get the labels for each sample
train_labels = [data["labels"].cpu().item() for data in train_set]
eval_labels = [data["labels"] for data in eval_set]
test_labels = [data["labels"] for data in test_set]

# Count the number of samples for each class in the train set
label_counts = Counter(train_labels)

# Calculate the weight for each sample
weights = [1.0 / label_counts[label] for label in train_labels]

# Create a weighted random sampler for the train set
sampler = WeightedRandomSampler(weights, len(weights))

# Set up a dataloader for the train set
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=256, sampler=sampler)
eval_dataloader = torch.utils.data.DataLoader(eval_set, batch_size=256, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False)

## Unbalanced

In [9]:
# Split the dataset into train and test
train_set, test_set = torch.utils.data.random_split(bert_multilingual_last_layer_dataset, [0.8, 0.2])
test_set, eval_set = torch.utils.data.random_split(test_set, [0.5, 0.5])


# Set up a dataloader for the train set
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True)
eval_dataloader = torch.utils.data.DataLoader(eval_set, batch_size=256, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False)

# Train

## Loss functions

In [10]:
# Set up a cross-entropy loss function
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
#wrong, in the log there should always be prob be fed in, otherwise large numbers just give best result
def loss_custom(outputs, labels, epsilon = 1e-12):
  outputs = outputs + 0.5
  outputs = torch.clip(outputs, epsilon, 1. - epsilon)                                      
  N = outputs.shape[0]
  ce = -torch.sum(soft_labels*torch.log(outputs))
  return ce/N


In [ ]:
loss_l1 = torch.nn.L1Loss()

In [ ]:
def wasserstein_loss(y_pred, y_true):
    return torch.mean(y_true * y_pred)


In [ ]:
train_labels = [data["labels"].cpu().item() for data in train_set]
label_counts = Counter(train_labels)
class_weights = [label_counts[label] / len(train_set) for label in label_counts]

loss_balanced = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weights).to(device))

## Model

In [52]:
import torch
from tqdm import tqdm_notebook

# Create a list of 10 linear layers
num_lin_layers = 5
lin_layers = [torch.nn.Linear(bert_multilingual_last_layer_dataset[0]["output"].shape[-1], 2) for _ in range(num_lin_layers)]

# Move the linear layers to the designated device
for lin_layer in lin_layers:
    lin_layer = lin_layer.to(device)

# Set up an Adam optimizer for each linear layer
optimizers = [torch.optim.Adam(lin_layer.parameters()) for lin_layer in lin_layers]



## Standard train

In [53]:
import matplotlib.pyplot as plt

# Set up a figure with subplots for each linear layer
#fig, axs = plt.subplots(len(lin_layers), sharex=True, figsize=(10,10))

# Train the linear layers
for i, lin_layer in enumerate(lin_layers):
    # Set the current linear layer to training mode
    lin_layer.train()
    
    # Set the current optimizer
    optimizer = optimizers[i]
    
    # Set up lists to store the losses for each epoch
    epoch_losses = []
    epoch_accuracies = []

    # Set the patience for early stopping
    patience = 2
    
    # Set the best loss to a large value
    best_accuracy = 10
    
    # Set the count of epochs without improvement to zero
    no_improvement_count = 0
    
    # Train the current linear layer for multiple epochs
    for epoch in range(10):
        # Set up a running loss for the epoch
        running_loss = 0.0
        running_correct_count = 0
        
        # Iterate through the train dataloader
        for batch in tqdm_notebook(train_dataloader):
            # Move the inputs and labels to the designated device
            inputs, hard_labels, soft_labels  = batch['output'].to(device), batch['labels'].to(device), batch['soft_labels']#.to(device)
            #num_annotations, annotations =  batch['num_annotations'].to(device), batch['annotations'].to(device)
            soft_labels = torch.cat([batch["soft_labels"]["0"][0].unsqueeze(0), batch["soft_labels"]["1"][0].unsqueeze(0)], dim=0).T.to(device)


            optimizer.zero_grad()

            outputs = lin_layer(inputs)

            loss = loss_fn(outputs, soft_labels)
            #loss  = loss_l1(outputs, soft_labels)

            loss.backward()

            optimizer.step()

            running_loss += loss.item()
            running_correct_count += torch.sum(torch.argmax(outputs, dim=1) == hard_labels).item()

        

        # Calculate the average loss and accuracy for the epoch
        avg_loss = running_loss / len(train_dataloader)
        avg_accuracy = running_correct_count / len(train_set)
        
        # Append the loss and accuracy for the epoch to the lists
        epoch_losses.append(avg_loss)
        epoch_accuracies.append(avg_accuracy)
        
        # Plot the losses for the epoch
        #axs[i].plot(epoch_losses)
        #plt.show()

        eval_accuracy = evaluate_ce_score_pytorch(lin_layer, eval_dataloader)
        
        # Check if the current loss is the best loss
        if eval_accuracy < best_accuracy:
            # Update the best accuracy
            best_accuracy = eval_accuracy
            
            # Reset the count of epochs without improvement
            no_improvement_count = 0
        
        else:
            # Increment the count of epochs without improvement
            no_improvement_count += 1
        
        # Check if the count of epochs without improvement has reached the patience
        if no_improvement_count >= patience:
            print(f'Linear Layer {i} has not improved for {patience} epochs, stopping training.')
            break





<ipython-input-53-d410518b1c5b>:34: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(train_dataloader):


  0%|          | 0/33 [00:00<?, ?it/s]

<ipython-input-4-b622ac5f4020>:73: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

...

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Linear Layer 4 has not improved for 2 epochs, stopping training.


## Pure annotations train

In [57]:
def map_elements(lst):
    if any(x < 0 for x in lst):
        return [1 if x in [-3, -2, -1] else 0 for x in lst]
    else:
        return lst

In [58]:
from pandas.core.indexes.numeric import annotations
import matplotlib.pyplot as plt
import random

# Set up a figure with subplots for each linear layer
#fig, axs = plt.subplots(len(lin_layers), sharex=True, figsize=(10,10))

# Train the linear layers
for i, lin_layer in enumerate(lin_layers):
    #each layer simulates one annotator
    ann = i % 6 

    # Set the current linear layer to training mode
    lin_layer.train()
    
    # Set the current optimizer
    optimizer = optimizers[i]
    
    # Set up lists to store the losses for each epoch
    epoch_losses = []
    epoch_accuracies = []

    # Set the patience for early stopping
    patience = 1
    
    # Set the best loss to a large value
    best_accuracy = 10
    
    # Set the count of epochs without improvement to zero
    no_improvement_count = 0
    
    # Train the current linear layer for multiple epochs
    for epoch in range(10):
        # Set up a running loss for the epoch
        running_loss = 0.0
        running_correct_count = 0
        
        # Iterate through the train dataloader
        for batch in tqdm_notebook(train_dataloader):
            # Move the inputs and labels to the designated device
            inputs = batch['output'].to(device)

            # generate a list of lists in the form [batch][annotators]
            annotations_list = [list(map(int, string.split(','))) for string in batch['annotations']]

            # Convert annotations that are not 0 or 1 (note this is not rly right probably, TODO)
            annotations_list = [map_elements(lst) for lst in annotations_list]

            # generate indices, such that the index in the range of the number of annotations
            indices = [ann % len(x) for x in annotations_list]

            # get the corresponding label for the annotator chosen
            labels = [x[y] for x,y in zip(annotations_list, indices)]

            # cast to tensors
            labels = torch.tensor(labels, device = device)

            #soft_labels = torch.cat([batch["soft_labels"]["0"][0].unsqueeze(0), batch["soft_labels"]["1"][0].unsqueeze(0)], dim=0).T.to(device)

            optimizer.zero_grad()

            outputs = lin_layer(inputs)

            loss = loss_fn(outputs, labels)
            #loss  = loss_l1(outputs, soft_labels)

            loss.backward()

            optimizer.step()

            running_loss += loss.item()
            #running_correct_count += torch.sum(torch.argmax(outputs, dim=1) == hard_labels).item()

        

        # Calculate the average loss and accuracy for the epoch
        avg_loss = running_loss / len(train_dataloader)
        avg_accuracy = running_correct_count / len(train_set)
        
        # Append the loss and accuracy for the epoch to the lists
        epoch_losses.append(avg_loss)
        epoch_accuracies.append(avg_accuracy)
        
        # Plot the losses for the epoch
        #axs[i].plot(epoch_losses)
        #plt.show()

        eval_accuracy = evaluate_ce_score_pytorch(lin_layer, eval_dataloader)
        
        # Check if the current loss is the best loss
        if eval_accuracy < best_accuracy:
            # Update the best accuracy
            best_accuracy = eval_accuracy
            
            # Reset the count of epochs without improvement
            no_improvement_count = 0
        
        else:
            # Increment the count of epochs without improvement
            no_improvement_count += 1
        
        # Check if the count of epochs without improvement has reached the patience
        if no_improvement_count >= patience:
            print(f'Linear Layer {i} has not improved for {patience} epochs, stopping training.')
            break





<ipython-input-58-78b680d91e98>:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(train_dataloader):


  0%|          | 0/33 [00:00<?, ?it/s]

<ipython-input-4-b622ac5f4020>:73: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

...

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Linear Layer 4 has not improved for 1 epochs, stopping training.


# Model combination

## Trainable combination layer

In [ ]:
class EnsembleModel(torch.nn.Module):

    def __init__(self, lin_layers, new_lin_layer):
        super(EnsembleModel, self).__init__()

        self.lin_layers = lin_layers
        self.new_lin_layer = new_lin_layer


    def forward(self, x):
      other_lin_layer_outputs = [torch.nn.functional.relu(lin_layer(x)) for lin_layer in self.lin_layers]
      # Concatenate the outputs
      concatenated_outputs = torch.cat(other_lin_layer_outputs, dim=1)
      new_outputs = self.new_lin_layer(concatenated_outputs)
      return new_outputs

# Create a new linear layer that takes as input the output of all other linear layers
new_lin_layer = torch.nn.Linear(num_lin_layers*2, 2)

# Move the new linear layer to the designated device
new_lin_layer = new_lin_layer.to(device)

ensemble_model = EnsembleModel(lin_layers, new_lin_layer)

In [ ]:
# Set up an Adam optimizer for the new linear layer
new_optimizer = torch.optim.Adam(new_lin_layer.parameters())

# Train the new linear layer

# Set the new linear layer to training mode
new_lin_layer.train()

# Set up lists to store the losses for each epoch
new_epoch_losses = []
new_epoch_accuracies = []

# Set the patience for early stopping
new_patience = 1

# Set the best loss to a large value
new_best_accuracy = 10

# Set the count of epochs without improvement to zero
new_no_improvement_count = 0

# Train the new linear layer for multiple epochs
for new_epoch in range(10):
    # Set up a running loss for the epoch
    new_running_loss = 0.0
    new_running_correct_count = 0

    # Iterate through the train dataloader
    for new_batch in tqdm_notebook(train_dataloader):
        # Move the inputs and labels to the designated device
        new_inputs, new_labels = new_batch['output'].to(device), new_batch['labels'].to(device)

        new_optimizer.zero_grad()

        # Get the output of all other linear layers
        other_lin_layer_outputs = [torch.nn.functional.relu(lin_layer(new_inputs)) for lin_layer in lin_layers]

        # Concatenate the outputs
        concatenated_outputs = torch.cat(other_lin_layer_outputs, dim=1)

        new_outputs = new_lin_layer(concatenated_outputs)

        new_loss = loss_fn(new_outputs, new_labels)

        new_loss.backward()

        new_optimizer.step()

        new_running_loss += new_loss.item()
        new_running_correct_count += torch.sum(torch.argmax(new_outputs, dim=1) == new_labels).item()

    # Calculate the average loss and accuracy for the epoch
    new_avg_loss = new_running_loss / len(train_dataloader)
    new_avg_accuracy = new_running_correct_count / len(train_set)

    # Append the loss and accuracy for the epoch to the lists
    new_epoch_losses.append(new_avg_loss)
    new_epoch_accuracies.append(new_avg_accuracy)

    new_eval_accuracy = evaluate_ce_score_pytorch(ensemble_model, eval_dataloader)

    # Check if the current loss is the best loss
    if new_eval_accuracy < new_best_accuracy:
        # Update the best accuracy
        new_best_accuracy = new_eval_accuracy
        # Reset the count of epochs without improvement
        new_no_improvement_count = 0

    else:
        # Increment the count of epochs without improvement
        new_no_improvement_count += 1

# Check if the count of epochs without improvement has reached the patience
    if new_no_improvement_count >= new_patience:
        print("Early stopping at epoch {} with accuracy {}".format(new_epoch+1, new_best_accuracy))
        break




<ipython-input-20-9b520d3c3148>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for new_batch in tqdm_notebook(train_dataloader):


  0%|          | 0/33 [00:00<?, ?it/s]

<ipython-input-3-b622ac5f4020>:73: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
r_ce = evaluate_ce_score_pytorch(ensemble_model, test_dataloader)
r_ce

<ipython-input-3-b622ac5f4020>:73: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):


  0%|          | 0/5 [00:00<?, ?it/s]

tensor(0.5908, device='cuda:0')

## STD combination

In [59]:
class StdModel(torch.nn.Module):

    def __init__(self, lin_layers, new_lin_layer):
        super(StdModel, self).__init__()
        self.lin_layers = lin_layers
        self.new_lin_layer = new_lin_layer


    def forward(self, x):
      other_lin_layer_outputs = [torch.nn.functional.relu(lin_layer(x)[:, 0].unsqueeze(1)) for lin_layer in lin_layers]
      concatenated_outputs_lin = torch.cat(other_lin_layer_outputs, dim=1)
      std = torch.std(concatenated_outputs_lin, dim = 1)
      concatenated_outputs = torch.cat([concatenated_outputs_lin, std.unsqueeze(1)], dim=1)
      new_outputs = self.new_lin_layer(concatenated_outputs)
      return new_outputs

# Create a new linear layer that takes as input the output of all other linear layers
new_lin_layer = torch.nn.Linear(num_lin_layers*1 + 1, 2)

# Move the new linear layer to the designated device
new_lin_layer = new_lin_layer.to(device)

std_model = StdModel(lin_layers, new_lin_layer)

In [ ]:
# Set up an Adam optimizer for the new linear layer
new_optimizer = torch.optim.Adam(new_lin_layer.parameters())

# Train the new linear layer

# Set the new linear layer to training mode
new_lin_layer.train()

# Set up lists to store the losses for each epoch
new_epoch_losses = []
new_epoch_accuracies = []

# Set the patience for early stopping
new_patience = 2

# Set the best loss to a large value
new_best_accuracy = 10

# Set the count of epochs without improvement to zero
new_no_improvement_count = 0

# Train the new linear layer for multiple epochs
for new_epoch in range(10):
    # Set up a running loss for the epoch
    new_running_loss = 0.0
    new_running_correct_count = 0

    # Iterate through the train dataloader
    for new_batch in tqdm_notebook(train_dataloader):
        # Move the inputs and labels to the designated device
        new_inputs, new_hard_labels = new_batch['output'].to(device), new_batch['labels'].to(device)
        new_soft_labels = torch.cat([new_batch["soft_labels"]["0"][0].unsqueeze(0), new_batch["soft_labels"]["1"][0].unsqueeze(0)], dim=0).T.to(device)


        new_optimizer.zero_grad()

        new_outputs = std_model(new_inputs)

        new_loss = loss_fn(new_outputs, new_soft_labels)

        new_loss.backward()

        new_optimizer.step()

        new_running_loss += new_loss.item()
        new_running_correct_count += torch.sum(torch.argmax(new_outputs, dim=1) == new_hard_labels).item()

    # Calculate the average loss and accuracy for the epoch
    new_avg_loss = new_running_loss / len(train_dataloader)
    new_avg_accuracy = new_running_correct_count / len(train_set)

    # Append the loss and accuracy for the epoch to the lists
    new_epoch_losses.append(new_avg_loss)
    new_epoch_accuracies.append(new_avg_accuracy)

    new_eval_accuracy = evaluate_ce_score_pytorch(std_model, eval_dataloader)

    # Check if the current loss is the best loss
    if new_eval_accuracy < new_best_accuracy:
        # Update the best accuracy
        new_best_accuracy = new_eval_accuracy
        # Reset the count of epochs without improvement
        new_no_improvement_count = 0

    else:
        # Increment the count of epochs without improvement
        new_no_improvement_count += 1

# Check if the count of epochs without improvement has reached the patience
    if new_no_improvement_count >= new_patience:
        print("Early stopping at epoch {} with accuracy {}".format(new_epoch+1, new_best_accuracy))
        break


In [61]:
evaluate_ce_score_pytorch(std_model, test_dataloader)


<ipython-input-4-b622ac5f4020>:73: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):


  0%|          | 0/5 [00:00<?, ?it/s]

tensor(0.6073, device='cuda:0')

## Average

In [13]:
class Avg_Model(torch.nn.Module):
    def __init__(self, lin_layers):
        super(Avg_Model, self).__init__()
        self.lin_layers = lin_layers
        
    def forward(self, x):
        # Feed the input through all linear layers
        outputs = []
        for lin_layer in self.lin_layers:
            outputs.append(lin_layer(x))
            
        # Calculate the average of the output of the linear layers
        avg_output = sum(outputs) / len(outputs)
        
        return avg_output


In [14]:
avg_model = Avg_Model(lin_layers).to(device)

In [15]:
r_ce = evaluate_ce_score_pytorch(avg_model, test_dataloader)
r_ce

<ipython-input-4-b622ac5f4020>:73: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):


  0%|          | 0/5 [00:00<?, ?it/s]

tensor(0.5833, device='cuda:0')

In [16]:
evaluate_bce_score_pytorch(avg_model, test_dataloader)

<ipython-input-4-b622ac5f4020>:56: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):


  0%|          | 0/5 [00:00<?, ?it/s]

tensor(0.5833, device='cuda:0')

# Results

|loss fun     | patience    | Labels      |Balancing    |num_layers   |combination  | ce 
| ----------- | ----------- | ----------- | ----------- | ----------- | ----------- |------------
| Ce loss     | 1           |soft         |None         |5            |avg          |**0.5649**
| Ce loss     | 2           |hard         |None         |5            |avg          |0.5806
| Ce loss     | 1           |hard         |None         |5            |avg          |0.5700
|Wasserstein  | 1           |soft         |None         |5            |avg          |0.6909
|L1           | 1           |soft         |None         |5            |avg          |0.5928
| Ce loss     | 1           |hard         |resampling   |5            |avg          |0.6970
| Ce loss     | 1           |hard         |weighted     |5            |avg          |0.7012
| Ce loss     | 1           |hard,ann     |None         |5            |avg          |0.7905
| Ce loss     | 1           |hard         |None         |10           |avg          |0.5833
| Ce loss     | 2           |hard         |None         |5            |lin_lay(P=2) |0.5850
| Ce loss     | 1           |hard,ann     |None         |5            |lin_lay(P=1) |0.5908
| Ce loss     | 2           |soft         |None         |5            |std          |0.5777

I am not sure why ce is suddenly 0.5..., it was 0.4... yesterday. Also baseline that spits out 0.5,0.5 everywhere was 0.37 yesterday and is now 0.69. No idea what changed

ann means that each layer simulated one annotator

lin_lay is the new_linear layer combination, with p indicating the patience

In [ ]:
#run.finish()

# Alternatives

In [ ]:
# # Set up a dataloader for the test set
# test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False)

# # Evaluate the linear layers on the test set
# for i, lin_layer in enumerate(tqdm_notebook(lin_layers, desc='Evaluating Linear Layers')):
#     # Set the current linear layer to evaluation mode

#     lin_layer.eval()
    
#     # Set up a running loss for the evaluation
#     running_loss = 0.0
    
#     # Set up a running correct count for the evaluation
#     running_correct_count = 0
    
#     # Iterate through the test dataloader
#     for inputs, labels in test_dataloader:
#         # Move the inputs and labels to the designated device
#         inputs = inputs.to(device)
#         labels = labels.to(device)
        
#         # Forward pass
#         with torch.no_grad():
#             outputs = lin_layer(inputs)
#             loss = loss_fn(outputs, labels)
        
#         # Update the running loss and correct count
#         running_loss += loss.item()
#         running_correct_count += torch.sum(torch.argmax(outputs, dim=1) == labels).item()
    
#     # Calculate the average loss and accuracy for the evaluation
#     avg_loss = running_loss / len(test_dataloader)
#     avg_accuracy = running_correct_count / len(test_set)
    
#     print(f'Linear Layer {i}: loss = {avg_loss}, accuracy = {avg_accuracy}')

In [ ]:
# import pandas as pd
# df = pd.DataFrame.from_records(bert_multilingual_last_layer_dataset)
# df

In [ ]:
# def evaluate_ce_score_ensemble(lin_layers, new_lin_layer, eval_dataloader):
#   criterion = torch.nn.CrossEntropyLoss(reduction="sum")
#   total = 0
#   ce = 0
#   for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
#     inputs,  hard_labels = batch["output"].to(device), batch["labels"].to(device)
#     soft_labels = torch.cat([batch["soft_labels"]["0"][0].unsqueeze(0), batch["soft_labels"]["1"][0].unsqueeze(0)], dim=0).T.to(device)
#     with torch.no_grad():
#       other_lin_layer_outputs = [torch.nn.functional.relu(lin_layer(inputs)) for lin_layer in lin_layers]
#       # Concatenate the outputs
#       concatenated_outputs = torch.cat(other_lin_layer_outputs, dim=1)
#       new_outputs = new_lin_layer(concatenated_outputs)

#     logits = outputs
#     probabilities = logits                         
#     N = probabilities.shape[0]
#     total+=N
#     ce += criterion(probabilities, soft_labels)
#   return ce/total